### Dependencies

In [1]:
# !pip install matplotlib tensorflow tensorflow_addons tensorflow_datasets imageio

In [2]:
# !pip install pydot

### Setup

In [3]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
from tensorflow.keras import Model
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib

from IPython import display

### Dataset

In [52]:
# train_ds = tf.keras.utils.image_dataset_from_directory(
# #   "/home/tony/TO_BE_REMOVED/celeba_data/imgs/",
#   "/Users/anthonylaw/Desktop/Endless/GAN-devel/mnist_ds/mnist_jpg/training",
#   seed=123,
#   image_size=(32, 32),
#   batch_size=16)

In [53]:
# for i_b, l_b in train_ds:
#     print(i_b.shape)
#     print(tf.image.rgb_to_grayscale(i_b).shape)

#### Note: 

Images should be normalized to [-1,1] ***(Done in Arch)***

### Generator network

#### Note:

Modify the network size for deployment

In [4]:
class Generator(Model):

    def __init__(self, noise_dim, image_shape, num_channel):
        super().__init__()
        
        assert len(image_shape) == 2
        assert image_shape[0]%8 == 0
        assert image_shape[1]%8 == 0
        
        self.noise_dim = noise_dim
        self.image_shape = image_shape
        self.num_channel = num_channel
        self.kernel_size = 3

        self.lr_d = layers.ReLU()
        self.lr_c1 = layers.ReLU()
        self.lr_c2 = layers.ReLU()
        self.lr_c3 = layers.ReLU()
        
        self.init_dense = layers.Dense(image_shape[0]/8.0*image_shape[1]/8.0*64,
                               use_bias=False, input_shape=(None, self.noise_dim))
        
        self.init_reshape = layers.Reshape((int(image_shape[0]/8.0), int(image_shape[1]/8.0), 64))
        
        self.conv2dT1 = layers.Conv2DTranspose(64, (self.kernel_size, self.kernel_size),
                                               strides=(1, 1), padding='same')
        self.conv2dT2 = layers.Conv2DTranspose(32, (self.kernel_size, self.kernel_size),
                                               strides=(2, 2), padding='same')
        self.conv2dT3 = layers.Conv2DTranspose(16, (self.kernel_size, self.kernel_size),
                                               strides=(2, 2), padding='same')
        self.conv2dTactv = layers.Conv2DTranspose(self.num_channel, (self.kernel_size, self.kernel_size),
                                               strides=(2, 2), padding='same', activation='tanh')

    def call(self, noise_vec):

        init_vec = self.lr_d(self.init_dense(noise_vec))
                
        reshaped = self.init_reshape(init_vec)
        
        convt1 = self.lr_c1(self.conv2dT1(reshaped))
        
        convt2 = self.lr_c2(self.conv2dT2(convt1))
                         
        convt3 = self.lr_c3(self.conv2dT3(convt2))
            
        out = self.conv2dTactv(convt3)

        return out
    
    def build_graph(self):
        x = layers.Input(shape=(self.noise_dim,))
        
        return Model(inputs=x, outputs=self.call(x))
        

#### Testing

In [5]:
# g1 = Generator(10, (32, 32), 3)

In [6]:
# g1.kernel_size

In [7]:
# noise_input = tf.random.normal((5, 10))
# print(noise_input.shape)
# pics1 = g1(noise_input)
# print(pics1.shape)
# # plt.imshow(pics1[-1, :, :, :], cmap='gray')

In [8]:
# tf.keras.utils.plot_model(g1.build_graph(), show_shapes=True)

In [9]:
# g1.build_graph().summary()

### Discriminator

In [3]:
class Discriminator(Model):

    def __init__(self, image_shape, num_channel):
        super().__init__()
        
        assert len(image_shape) == 2
        assert image_shape[0]%8 == 0
        assert image_shape[1]%8 == 0
        
        self.image_shape = image_shape
        self.num_channel = num_channel
        self.kernel_size = 3

        self.lr_c1 = layers.LeakyReLU()
        self.lr_c2 = layers.LeakyReLU()
        self.lr_c3 = layers.LeakyReLU()
        self.flatten = layers.Flatten()
        
        self.conv2d1 = layers.Conv2D(16, (self.kernel_size, self.kernel_size),
                                        strides=(2, 2), padding='same',
                                        input_shape=(None, self.image_shape[0],
                                        self.image_shape[1], self.num_channel))
        self.conv2d2 = layers.Conv2D(32, (self.kernel_size, self.kernel_size),
                                               strides=(2, 2), padding='same')
        self.conv2d3 = layers.Conv2D(64, (self.kernel_size, self.kernel_size),
                                               strides=(2, 2), padding='same')
        self.dense_actv = layers.Dense(64,
                                      )
#                                        activation="sigmoid")
        
    def call(self, img_input):
        
        conv1 = self.lr_c1(self.conv2d1(img_input))

        conv2 = self.lr_c2(self.conv2d2(conv1))
                         
        conv3 = self.lr_c3(self.conv2d3(conv2))

        flat = self.flatten(conv3)
        
        out = self.dense_actv(flat)
        
        return out

    def build_graph(self):
        x = layers.Input(shape=(self.image_shape[0],
                                        self.image_shape[1], self.num_channel))
        
        return Model(inputs=x, outputs=self.call(x))
        

#### Testing

In [10]:
# d1 = Discriminator((32,32), 3)
# g2 = Generator(10, (32,32), 3)
# d1.kernel_size

In [11]:
# noise_input = tf.random.normal((5, 10))
# pics2 = g1(noise_input)
# # plt.imshow(pics2[-1, :, :, 0], cmap='gray')
# print(pics2.shape)

In [12]:
# deci = d1(pics2)
# deci

In [13]:
# d1.build_graph().summary()

### CramerDCGAN

In [49]:
EPSILON = 1e-16

class DCGAN:
    
    def __init__(self, dataset_path, image_shape, num_channel, noise_latent_dim, disc_update_multi=5, 
                 batch_size=64, lr=3e-4, gp_lam = 10.0, div_lam = 0.1):
        assert len(image_shape) == 2
        
        self.image_shape = image_shape
        self.num_channel = num_channel
        self.noise_latent_dim = noise_latent_dim
        self.batch_size, self.gp_lam = batch_size, gp_lam
        self.disc_update_multi = disc_update_multi
        self.num_img_prog_monit = 16
        self.div_lam = div_lam
        
        if not dataset_path==None:
            self.dataset = tf.keras.utils.image_dataset_from_directory(
                                  dataset_path,
                                  seed=123,
                                  image_size=self.image_shape,
                                  batch_size=self.batch_size)
            
        else:
            print("WARNING: Dataset not loaded, Model in Generator mode")
        # NOTE: Dataset must be processed differently for different source and applications
        
        self.g = Generator(self.noise_latent_dim, self.image_shape, self.num_channel)
        self.d = Discriminator(self.image_shape, self.num_channel)
        
        self.g_opt = tf.keras.optimizers.Adam(lr)
        self.d_opt = tf.keras.optimizers.Adam(lr)
        
        self.g_seed = tf.random.normal((self.num_img_prog_monit, self.noise_latent_dim))

    def cramer_loss(self, d_x_data, d_g_z_1, d_g_z_2, x_it):
        
        crit_r = tf.math.add(tf.math.sqrt(tf.reduce_sum(tf.math.add(d_x_data, -d_g_z_2)**2, axis = 1)+EPSILON),
                   -tf.math.sqrt(tf.reduce_sum(d_x_data**2, axis = 1)+EPSILON))
        crit_g_1 = tf.math.add(tf.math.sqrt(tf.reduce_sum(tf.math.add(d_g_z_1, -d_g_z_2)**2, axis = 1)+EPSILON),
                   -tf.math.sqrt(tf.reduce_sum(d_g_z_1**2, axis = 1)+EPSILON))

        L_srg = tf.math.add(crit_r, -crit_g_1)
        
        with tf.GradientTape() as t_gp:
            t_gp.watch(x_it)
            d_it = self.d(x_it)
            crit_it = tf.math.add(tf.math.sqrt(tf.reduce_sum(tf.math.add(d_it, -d_g_z_2)**2, axis = 1)+EPSILON),
                   -tf.math.sqrt(tf.reduce_sum(tf.math.add(d_it, -d_x_data)**2, axis = 1)+EPSILON))
            
        gp_grad = t_gp.gradient(crit_it, x_it)
        l2n_gp = tf.math.sqrt(tf.reduce_sum(gp_grad**2, axis = [1,2,3])+EPSILON)
        L_gp = self.gp_lam*((l2n_gp-1.0)**2)


        # g_loss
        L_g = tf.reduce_mean(L_srg)
        
        # d_loss
        L_d = tf.reduce_mean(-L_srg + L_gp)

        return L_g, L_d
    
    def div_loss(self, g_z1, g_z2):

        L_div = tf.math.sqrt(tf.reduce_sum(tf.math.add(g_z1, -g_z2)**2, axis = [1,2,3])+EPSILON)

        return tf.reduce_mean(L_div)
    
    @tf.function
    def update(self, imgs, update_gen=True):
        noise_input1 = tf.random.normal((imgs.shape[0], self.noise_latent_dim))
        noise_input2 = tf.random.normal((imgs.shape[0], self.noise_latent_dim))
        
        with tf.GradientTape() as g_tape, tf.GradientTape() as d_tape:
            g_z_1 = self.g(noise_input1)
            g_z_2 = self.g(noise_input2)
            
            d_x_data = self.d(imgs)
            d_g_z_1 = self.d(g_z_1)
            d_g_z_2 = self.d(g_z_2)
            
            epsi = tf.random.uniform([imgs.shape[0], 1, 1, 1], 0.0, 1.0)
            x_it = tf.math.add(epsi*imgs, (1.0-epsi)*g_z_1)
            g_loss, d_loss = self.cramer_loss(d_x_data, d_g_z_1, d_g_z_2, x_it)
            div_loss = self.div_loss(g_z_1, g_z_2)
            g_loss = g_loss-self.div_lam*div_loss

        if update_gen:
            grad_g = g_tape.gradient(g_loss, self.g.trainable_variables)
            grad_d = d_tape.gradient(d_loss, self.d.trainable_variables)

            self.g_opt.apply_gradients(zip(grad_g, self.g.trainable_variables))
            self.d_opt.apply_gradients(zip(grad_d, self.d.trainable_variables))
        else:
            grad_d = d_tape.gradient(d_loss, self.d.trainable_variables)
            self.d_opt.apply_gradients(zip(grad_d, self.d.trainable_variables))
            
        return g_loss, d_loss
        
    def train(self, epochs=250, train_moni_path=None, g_path=None, d_path=None):
        num_training = 0
        for epo in range(epochs):
            g_losses = []
            d_losses = []
            for img_b, l_b in self.dataset:
                if self.num_channel == 1 and img_b.shape[-1] == 3:
                    img_b = tf.image.rgb_to_grayscale(img_b)
                img_b = (img_b-127.5)/127.5    
                if num_training%self.disc_update_multi == 0:
                    g_l, d_l = self.update(img_b, True)
                    g_losses.append(g_l.numpy())
                    d_losses.append(d_l.numpy())
                    
                else:
                    g_l, d_l = self.update(img_b, False)
                    d_losses.append(d_l.numpy())
                    
                num_training = (num_training+1)%self.disc_update_multi
                
            print("Epoch {:04d}".format(epo), "Generator Avg. Loss: ", np.mean(g_losses), 
                  ", Discriminator Avg. Loss: ",  np.mean(d_losses), flush=True)
            
            if not g_path==None and not d_path==None:
                self.save_weights(g_path, d_path)
                
            if not train_moni_path == None:
                self.monitor_progress(epo, train_moni_path)
            
    def monitor_progress(self, epo, path):
        pics = self.g(self.g_seed)
        
        fig = plt.figure(figsize=(4,4))
        for i in range(pics.shape[0]):
            plt.subplot(4,4,i+1)
            if self.num_channel == 1:
                plt.imshow(pics[i,:,:,0], cmap='gray')
            else:   
                plt.imshow(tf.cast(tf.math.round(pics[i,:,:,:]*127.5+127.5), tf.int32))
            plt.axis('off')
            
        plt.savefig(path+'/image_{:04d}.png'.format(epo))
#         plt.savefig('/home/tony/TO_BE_REMOVED/imgs/image_{:04d}.png'.format(epo))
        # NEEDS to be changed for machines
        
        plt.close('all')
        
    def save_weights(self, g_path, d_path):
        self.g.save_weights(g_path)
        print("Saved generator weights", flush=True)
        self.d.save_weights(d_path)
        print("Saved discriminator weights", flush=True)
    def load_weights(self, g_path, d_path):
        try:
            self.g.load_weights(g_path).expect_partial()
            print("Loaded generator weights", flush=True)
            self.d.load_weights(d_path).expect_partial()
            print("Loaded discriminator weights", flush=True)
        except ValueError:
            print("ERROR: Please make sure weights are saved as .ckpt", flush=True)
    
    def generate_samples(self, num_sam, path):
        sam_seed = tf.random.normal((num_sam, self.noise_latent_dim))
        sam_pics = self.g(sam_seed)
        
        dpi = matplotlib.rcParams['figure.dpi']
        
        for i in range(sam_pics.shape[0]):
            
            figsize = self.image_shape[1] / float(dpi), self.image_shape[0] / float(dpi)
            fig = plt.figure(figsize=figsize)
            ax = fig.add_axes([0, 0, 1, 1])
            ax.axis('off')
            
            if self.num_channel == 1:
                ax.imshow(sam_pics[i,:,:,0], cmap='gray', interpolation='nearest')
            else:   
                ax.imshow(tf.cast(tf.math.round(sam_pics[i,:,:,:]*127.5+127.5), tf.int32), interpolation='nearest')

            fig.savefig(path+'/image_{:04d}.png'.format(i), dpi=dpi, transparent=True)
            plt.close('all')
            
    def model_params(self):
        self.g.build_graph().summary()
        self.d.build_graph().summary()

#### Testing

#### Img save test

In [15]:
# dpi = matplotlib.rcParams['figure.dpi']

In [16]:
# im_data = plt.imread('/home/data_backup/data_bu/mnist_jpg/training/9/.vou/GAN-devel/model.png')
# height, width, _ = im_data.shape

In [17]:
# figsize = width / float(dpi), height / float(dpi)

In [50]:
# fig = plt.figure(figsize=figsize)
# ax = fig.add_axes([0, 0, 1, 1])

In [51]:
# ax.axis('off')

In [52]:
# ax.imshow(im_data, interpolation='nearest')

In [53]:
# fig.savefig('test.png', dpi=dpi, transparent=True)

#### Training test

In [57]:
# ds_path = "/home/data_backup/Downloads/archive/BreaKHis_v1/BreaKHis_v1/histology_slides/breast/train400Xpng"
ds_path = "/home/szorek/mnist/mnist_jpg"
moni_path = "/home/szorek/toxoplasmosis-research/samples/gan_moni"

In [58]:
dcgan1 = DCGAN(ds_path, (28, 28), 1, 32, disc_update_multi=5)

Found 60000 files belonging to 1 classes.


2023-03-22 19:35:39.727655: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-22 19:35:40.110526: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21388 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:65:00.0, compute capability: 8.6


In [22]:
# dcgan1.model_params()

In [59]:
dcgan1.train(20,moni_path)
# dcgan1.save_weights('./weights/g_test.ckpt', './weights/d_test.ckpt')

2023-03-22 19:36:04.441364: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-03-22 19:36:04.893817: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


Epoch 0000 Generator Avg. Loss:  7.784465 , Discriminator Avg. Loss:  -7.2073035
Epoch 0001 Generator Avg. Loss:  4.7792363 , Discriminator Avg. Loss:  -5.123457
Epoch 0002 Generator Avg. Loss:  3.6593077 , Discriminator Avg. Loss:  -4.1692286
Epoch 0003 Generator Avg. Loss:  3.1466382 , Discriminator Avg. Loss:  -3.7634003
Epoch 0004 Generator Avg. Loss:  3.1108131 , Discriminator Avg. Loss:  -3.3040166
Epoch 0005 Generator Avg. Loss:  1.4628533 , Discriminator Avg. Loss:  -2.3284109
Epoch 0006 Generator Avg. Loss:  1.0110052 , Discriminator Avg. Loss:  -2.0127344
Epoch 0007 Generator Avg. Loss:  0.90865517 , Discriminator Avg. Loss:  -1.0592635
Epoch 0008 Generator Avg. Loss:  -0.54232895 , Discriminator Avg. Loss:  -0.5342614
Epoch 0009 Generator Avg. Loss:  -0.24669757 , Discriminator Avg. Loss:  -0.87874955
Epoch 0010 Generator Avg. Loss:  0.44620368 , Discriminator Avg. Loss:  -1.511617
Epoch 0011 Generator Avg. Loss:  0.6013062 , Discriminator Avg. Loss:  -1.6124732
Epoch 0012 G

In [24]:
# dcgan1.load_weights('./weights/g_test.ckpt', './weights/d_test.ckpt')

In [25]:
# dcgan1.generate_samples(10, './samples/0')

In [26]:
# dcgan2 = DCGAN(ds_path, (32, 32), 1, 25, disc_update_multi=1)

In [27]:
# dcgan2.load_weights('./weights/g_test.ckpt', './weights/d_test.ckpt')

In [28]:
# dcgan2.train(20, './imgs')

In [29]:
# dcgan2.save_weights('./weights/g_test.ckpt', './weights/d_test.ckpt')

In [30]:
# dcgan1.load_weights('./weights/g_test.ckpt', './weights/d_test.ckpt')

In [31]:
# dcgan3 = DCGAN(None, (32, 32), 1, 25, disc_update_multi=1)

In [32]:
# dcgan3.load_weights('./weights/g_test.ckpt', './weights/d_test.ckpt')

In [33]:
# dcgan3.generate_samples(10, './samples/1')

### Pre-Act Resblk Generator

In [43]:
class Generator(Model):

    def __init__(self, noise_dim, image_shape, num_channel):
        super().__init__()
        
        assert len(image_shape) == 2
        assert image_shape[0]%4 == 0
        assert image_shape[1]%4 == 0
        
        self.noise_dim = noise_dim
        self.image_shape = image_shape
        self.num_channel = num_channel
        self.kernel_size = 3
        
        # add layer(multiple for plot clarification)
        self.add1 = layers.Add()
        self.add2 = layers.Add()
        
        # init layers
        self.init_dense = layers.Dense(image_shape[0]/4.0*image_shape[1]/4.0,
                               use_bias=False, input_shape=(None, self.noise_dim), activation='relu')
        
        self.init_reshape = layers.Reshape((int(image_shape[0]/4.0), int(image_shape[1]/4.0),1))
        
        # in conv 1x1
        self.in_conv11 = layers.Conv2D(64, (1, 1),
                                strides=(1, 1), padding='same')
        
        # resblk 1
        self.rb1_in1 = tfa.layers.InstanceNormalization(axis=-1)
        self.rb1_rl1 = layers.ReLU()
        self.rb1_conv2d1 = layers.Conv2D(64, (self.kernel_size, self.kernel_size),
                                               strides=(1, 1), padding='same')
        
        self.rb1_in2 = tfa.layers.InstanceNormalization(axis=-1)
        self.rb1_rl2 = layers.ReLU()
        self.rb1_conv2d2 = layers.Conv2D(64, (self.kernel_size, self.kernel_size),
                                               strides=(1, 1), padding='same')
        
        # upsample 1
        self.us1 = layers.Conv2DTranspose(32, (self.kernel_size, self.kernel_size),
                                               strides=(2, 2), padding='same')
        
        # resblk2
        self.rb2_in1 = tfa.layers.InstanceNormalization(axis=-1)
        self.rb2_rl1 = layers.ReLU()
        self.rb2_conv2d1 = layers.Conv2D(32, (self.kernel_size, self.kernel_size),
                                               strides=(1, 1), padding='same')
        
        self.rb2_in2 = tfa.layers.InstanceNormalization(axis=-1)
        self.rb2_rl2 = layers.ReLU()
        self.rb2_conv2d2 = layers.Conv2D(32, (self.kernel_size, self.kernel_size),
                                               strides=(1, 1), padding='same')
        
        # upsample 2
        self.us2 = layers.Conv2DTranspose(16, (self.kernel_size, self.kernel_size),
                                               strides=(2, 2), padding='same')
        
        # out conv 1x1
        self.out_conv11 = layers.Conv2D(self.num_channel, (1, 1),
                                        strides=(1, 1), padding='same', activation='tanh')

    def call(self, noise_vec):
        
        # in
        x = self.init_reshape(self.init_dense(noise_vec))
#         x = self.init_reshape(noise_vec)
        
        # conv 1x1
        x = self.in_conv11(x)
        
        # resblk 1
        x_res = self.rb1_conv2d1(self.rb1_rl1(self.rb1_in1(x)))
        x_res = self.rb1_conv2d2(self.rb1_rl2(self.rb1_in2(x_res)))
        x = self.add1([x, x_res])
        
        # upsample 1
        x = self.us1(x)
        
        # resblk 2
        x_res = self.rb2_conv2d1(self.rb2_rl1(self.rb2_in1(x)))
        x_res = self.rb2_conv2d2(self.rb2_rl2(self.rb2_in2(x_res)))
        x = self.add2([x, x_res])
        
        # upsample 2
        x = self.us2(x)
        
        # conv 1x1
        x = self.out_conv11(x)
        
        return x

    def build_graph(self):
        x = layers.Input(shape=(self.noise_dim,))
        
        return Model(inputs=x, outputs=self.call(x))
        

In [44]:
# g1 = Generator(10, (32, 32), 3)

In [45]:
# noise_input = tf.random.normal((5, 10))

In [46]:
# tf.keras.utils.plot_model(g1.build_graph(), show_shapes=True)

In [47]:
# g1.build_graph().summary()

### Pre-Act Resblk Discriminator

In [48]:
class Discriminator(Model):

    def __init__(self, image_shape, num_channel):
        super().__init__()
        
        assert len(image_shape) == 2
        
        self.image_shape = image_shape
        self.num_channel = num_channel
        self.kernel_size = 3
        
        # add layer(multiple for plot clarification)
        self.add1 = layers.Add()
        self.add2 = layers.Add()
        
        
        # in conv 1x1
        self.in_conv11 = layers.Conv2D(16, (1, 1),
                                        strides=(1, 1), padding='same',
                                        input_shape=(None, self.image_shape[0],
                                        self.image_shape[1], self.num_channel))
        
        # resblk 1 
        self.rb1_rl1 = layers.LeakyReLU()
        self.rb1_conv2d1 = layers.Conv2D(16, (self.kernel_size, self.kernel_size),
                                               strides=(1, 1), padding='same')
        
        self.rb1_rl2 = layers.LeakyReLU()
        self.rb1_conv2d2 = layers.Conv2D(16, (self.kernel_size, self.kernel_size),
                                               strides=(1, 1), padding='same')
        
        # downsample 1
        self.ds1 = layers.Conv2D(32, (self.kernel_size, self.kernel_size),
                                               strides=(2, 2), padding='same')
        
        # resblk 2
        self.rb2_rl1 = layers.LeakyReLU()
        self.rb2_conv2d1 = layers.Conv2D(32, (self.kernel_size, self.kernel_size),
                                               strides=(1, 1), padding='same')
        
        self.rb2_rl2 = layers.LeakyReLU()
        self.rb2_conv2d2 = layers.Conv2D(32, (self.kernel_size, self.kernel_size),
                                               strides=(1, 1), padding='same')
        
        # downsample 2
        self.ds2 = layers.Conv2D(8, (self.kernel_size, self.kernel_size),
                                               strides=(2, 2), padding='same')
        
        # reshape for linear act
        self.rs_out = layers.Reshape((int(self.image_shape[0]/4.0)*int(self.image_shape[1]/4.0)*8,))
        
        # out
        self.dense_actv = layers.Dense(8)

        
        
    def call(self, img_input):
        # in
        x = self.in_conv11(img_input)
        
        # resblk 1
        x_res = self.rb1_conv2d1(self.rb1_rl1(x))
        x_res = self.rb1_conv2d2(self.rb1_rl2(x_res))
        x = self.add1([x, x_res])
        
        # resblk 1 downsample
        x = self.ds1(x)
        
        # resblk 2
        x_res = self.rb2_conv2d1(self.rb2_rl1(x))
        x_res = self.rb2_conv2d2(self.rb2_rl2(x_res))
        x = self.add2([x, x_res])
        
        # resblk 2 downsample
        x = self.ds2(x)
        
        # reshape
        x = self.rs_out(x)
        
        # out
        x = self.dense_actv(x)

        return x
    
    def build_graph(self):
        x = layers.Input(shape=(self.image_shape[0],
                                        self.image_shape[1], self.num_channel))
        
        return Model(inputs=x, outputs=self.call(x))
        

In [40]:
# d1 = Discriminator((160, 120), 3)

In [41]:
# tf.keras.utils.plot_model(d1.build_graph(), show_shapes=True)

In [42]:
# d1.build_graph().summary()